In [13]:
#Very Simple Simulation


 import random
import numpy as np

class Passenger:
    def __init__(self, seat_number):
        self.seat_number = seat_number
        self.boarding_time = random.uniform(0, 1)  # Random boarding time
        self.num_bags = random.uniform (1,4)
     

class Aircraft:
    def __init__(self, num_seats):
        self.num_seats = num_seats
        self.passengers = [Passenger(i) for i in range(1, num_seats + 1)]
        self.passengers.sort(key=lambda p: p.seat_number)  # Sort passengers by seat number

    def back_to_front_boarding(self):
        boarding_order = [p for p in self.passengers]
        boarding_order.sort(key=lambda p: p.seat_number, reverse=True)
        boarding_time = 0
        for passenger in boarding_order:
            boarding_time += passenger.boarding_time
        return boarding_time

    def random_boarding(self):
        random.shuffle(self.passengers)
        boarding_time = sum(p.boarding_time for p in self.passengers)
        return boarding_time

def simulate_boarding(num_simulations, num_seats, boarding_type):
    boarding_times = []

    for _ in range(num_simulations):
        aircraft = Aircraft(num_seats)
        if boarding_type == "back_to_front":
            boarding_time = aircraft.back_to_front_boarding()
        elif boarding_type == "random":
            boarding_time = aircraft.random_boarding()
        else:
            raise ValueError("Invalid boarding type specified")
        boarding_times.append(boarding_time)
    
    average_boarding_time = np.mean(boarding_times)
    percentile_5 = np.percentile(boarding_times, 5)
    percentile_95 = np.percentile(boarding_times, 95)

    return average_boarding_time, percentile_5, percentile_95

if __name__ == "__main__":
    num_simulations = 1000
    num_seats = 150
    boarding_type = "random"  # You can change this to "random" or other strategies
    average_time, percentile_5, percentile_95 = simulate_boarding(num_simulations, num_seats, boarding_type)
    print(f"Average boarding time for {num_simulations} simulations ({boarding_type}): {average_time}")
    print(f"5th percentile boarding time: {percentile_5}")
    print(f"95th percentile boarding time: {percentile_95}")


Average boarding time for 1000 simulations (random): 74.98961604663397
5th percentile boarding time: 68.96801476508963
95th percentile boarding time: 80.76971306307864


In [44]:
import random
import numpy as np

class Passenger:
    def __init__(self, row, seat_position):
        self.row = row
        self.seat_position = seat_position
        self.walking_speed = max(0.4, min(random.gauss(0.8, 0.1), 1.2))  # Random walking speed around 0.8 m/s
        self.num_bags = random.choice([1, 1, 1, 2, 2, 3])  # Random number of bags (1, 2, or 3 with commonality of 1)
        self.bag_location = [random.choice(["Overhead", "Under_seat"]) for _ in range(self.num_bags)]  # Bag location for each bag
        self.boarding_time = self.calculate_boarding_time()

    def calculate_boarding_time(self):
        average_row_length = 0.85  # Average row length in meters
        time_to_reach_window = 15  # Additional time for window seat in seconds
        time_to_reach_middle = 5  # Additional time for middle seat in seconds

        if self.seat_position == "Aisle":
            time_to_seat = self.row * average_row_length / self.walking_speed
        elif self.seat_position == "Middle":
            # Check if there's already a passenger in the middle seat of the same row
            time_to_reach_middle = 0
            time_to_seat = self.row * average_row_length / self.walking_speed + time_to_reach_middle  # In seconds
        elif self.seat_position == "Window":
            # Check if there's already a passenger in the window seat of the same row
            time_to_reach_window = 0
            time_to_seat = self.row * average_row_length / self.walking_speed + time_to_reach_window  # In seconds

        # Calculate additional time for bags based on their location
        for bag_location in self.bag_location:
            if bag_location == "Overhead":
                time_to_seat += 45  # Extra time for placing a bag in the overhead compartment in seconds
            elif bag_location == "Under_seat":
                time_to_seat += 5  # Extra time for placing a bag under the seat in seconds

        return time_to_seat

class Aircraft:
    def __init__(self, num_rows):
        self.num_rows = num_rows
        self.passengers = []
        seat_positions = ["Window", "Middle", "Aisle"]
        for row in range(1, num_rows + 1):
            row_passengers = []
            for seat_position in seat_positions:
                for _ in range(2):  # Two passengers per seat position
                    passenger = Passenger(row, seat_position)
                    row_passengers.append(passenger)
            random.shuffle(row_passengers)  # Shuffle the passengers in the row
            self.passengers.extend(row_passengers)
        random.shuffle(self.passengers)  # Shuffle the passenger list for random boarding order

    def simulate_boarding(self):
        total_time = 0
        for i in range(1, len(self.passengers)):
            previous_passenger = self.passengers[i - 1]
            current_passenger = self.passengers[i]
            pace = min(previous_passenger.walking_speed, current_passenger.walking_speed)
            distance = 1  # Assuming 1 meter spacing between passengers
            time_to_reach_seat = distance / pace
            total_time += time_to_reach_seat + current_passenger.boarding_time
        return total_time

if __name__ == "__main__":
    num_simulations = 1000
    num_rows = 32  # Number of rows in the aircraft
    aircraft = Aircraft(num_rows)
    boarding_times = []

    for _ in range(num_simulations):
        total_boarding_time = aircraft.simulate_boarding()
        average_time = total_boarding_time / len(aircraft.passengers)
        boarding_times.append(total_boarding_time)

    # Calculate and print average, 5th percentile, and 95th percentile boarding times
    average_boarding_time = np.mean(boarding_times)
    percentile_5 = np.percentile(boarding_times, 5)
    percentile_95 = np.percentile(boarding_times, 95)

    print(f"Average boarding time for {num_simulations} simulations: {average_boarding_time/3600} hours")
    print(f"5th percentile boarding time: {percentile_5/3600} hours")
    print(f"95th percentile boarding time: {percentile_95/3600} hours")


Average boarding time for 1000 simulations: 3.2076220718570894 hours
5th percentile boarding time: 3.2076220718570894 hours
95th percentile boarding time: 3.2076220718570894 hours
